# GRIB Processing

Converting the GRIB files into something useful (NetCDF or zarr).

In [1]:
import cartopy.crs as ccrs
import numpy as np
import pygrib
import xarray as xr

In [2]:
gribs = pygrib.open("data/2132119000017")

In [3]:
set([(m.parameterName, m.shortName, m.units) for m in gribs.select(typeOfLevel="hybrid")])

{('100', 'unknown', 'unknown'),
 ('192', 'pmtc', '(10**-6 g) m**-3'),
 ('193', 'pmtf', '(10**-6 g) m**-3'),
 ('28', 'unknown', 'unknown'),
 ('29', 'unknown', 'unknown'),
 ('32', 'cc', '(0 - 1)'),
 ('82', 'unknown', 'unknown'),
 ('Cloud mixing ratio', 'clwmr', 'kg kg**-1'),
 ('Geopotential height', 'gh', 'gpm'),
 ('Graupel (snow pellets)', 'grle', 'kg kg**-1'),
 ('Mass density', 'unknown', 'unknown'),
 ('Pressure', 'pres', 'Pa'),
 ('Rain mixing ratio', 'rwmr', 'kg kg**-1'),
 ('Snow mixing ratio', 'snmr', 'kg kg**-1'),
 ('Specific humidity', 'q', 'kg kg**-1'),
 ('Temperature', 't', 'K'),
 ('Turbulent kinetic energy', 'tke', 'J kg**-1'),
 ('Vertical velocity (pressure)', 'w', 'Pa s**-1'),
 ('u-component of wind', 'u', 'm s**-1'),
 ('v-component of wind', 'v', 'm s**-1')}

In [4]:
massden = gribs[20]

In [5]:
massden.gridType

'lambert'

In [6]:
list(sorted(massden.keys()))

['Dx',
 'DxInMetres',
 'Dy',
 'DyInMetres',
 'GRIBEditionNumber',
 'LaD',
 'LaDInDegrees',
 'Latin1',
 'Latin1InDegrees',
 'Latin2',
 'Latin2InDegrees',
 'LoV',
 'LoVInDegrees',
 'NV',
 'Nx',
 'Ny',
 'PLPresent',
 'PVPresent',
 'alternativeRowScanning',
 'analDate',
 'angleSubdivisions',
 'average',
 'backgroundProcess',
 'binaryScaleFactor',
 'bitMapIndicator',
 'bitmapPresent',
 'bitsPerValue',
 'bottomLevel',
 'centre',
 'centreDescription',
 'cfName',
 'cfNameECMF',
 'cfVarName',
 'cfVarNameECMF',
 'changeDecimalPrecision',
 'codedValues',
 'dataDate',
 'dataRepresentationTemplateNumber',
 'dataTime',
 'day',
 'decimalPrecision',
 'decimalScaleFactor',
 'deleteCalendarId',
 'deleteLocalDefinition',
 'deletePV',
 'discipline',
 'distinctLatitudes',
 'distinctLongitudes',
 'editionNumber',
 'endStep',
 'forecastTime',
 'genVertHeightCoords',
 'generatingProcessIdentifier',
 'getNumberOfValues',
 'globalDomain',
 'grib2LocalSectionPresent',
 'grib2divider',
 'gridDefinitionDescription

In [7]:
massden.validDate, massden.validityDate, massden.validityTime

(datetime.datetime(2021, 11, 18, 12, 0), 20211118, 1200)

In [8]:
massden.forecastTime

17

In [9]:
massden.analDate

datetime.datetime(2021, 11, 17, 19, 0)

## Grid Setup

In [6]:
EARTH_EQUATORIAL_RADIUS_M = 6371229.0

In [7]:
def grid(grib_msg):
    central_latitude = grib_msg.LaDInDegrees
    central_longitude = grib_msg.LoVInDegrees
    standard_parallels = (
        grib_msg.Latin1InDegrees,
        grib_msg.Latin2InDegrees,
    )
    
    globe = ccrs.Globe(ellipse="sphere", semimajor_axis=EARTH_EQUATORIAL_RADIUS_M)
    lambert = ccrs.LambertConformal(
        central_latitude=central_latitude,
        central_longitude=central_longitude,
        standard_parallels=standard_parallels,
        globe=globe
    )
    
    start = [
        grib_msg.longitudeOfFirstGridPointInDegrees,
        grib_msg.latitudeOfFirstGridPointInDegrees,
    ]

    if start[0] > 180:
        start[0] -= 360

    dx = grib_msg.DxInMetres
    dy = grib_msg.DyInMetres

    nx = grib_msg.Nx
    ny = grib_msg.Ny
    
    x0, y0 = lambert.transform_point(start[0], start[1], ccrs.Geodetic())
    
    return (
        np.arange(nx) * dx + x0,
        np.arange(ny) * dy + y0,
    )

In [8]:
x, y = grid(massden)

In [9]:
lats, lons = massden.latlons()

In [10]:
xr.DataArray(
    massden.values,
    coords={
        "latitude": (["y", "x"], lats),
        "longitude": (["y", "x"], lons),
        "x": x,
        "y": y
    },
    dims=["y", "x"]
)

<xarray.DataArray (y: 1059, x: 1799)>
array([[0.164, 0.164, 0.162, ..., 0.018, 0.018, 0.018],
       [0.164, 0.164, 0.162, ..., 0.018, 0.018, 0.018],
       [0.164, 0.164, 0.168, ..., 0.018, 0.018, 0.018],
       ...,
       [0.002, 0.002, 0.002, ..., 0.032, 0.032, 0.032],
       [0.002, 0.002, 0.002, ..., 0.032, 0.032, 0.032],
       [0.002, 0.002, 0.002, ..., 0.032, 0.032, 0.032]])
Coordinates:
    latitude   (y, x) float64 21.14 21.15 21.15 21.16 ... 47.86 47.85 47.84
    longitude  (y, x) float64 -122.7 -122.7 -122.7 ... -60.99 -60.95 -60.92
  * x          (x) float64 -2.698e+06 -2.695e+06 ... 2.693e+06 2.696e+06
  * y          (y) float64 -1.587e+06 -1.584e+06 ... 1.584e+06 1.587e+06

In [11]:
gribs.close()

## Dataset Construction

In [12]:
def to_data_array(grib_msg): 
    x, y = grid(grib_msg)
    lats, lons = grib_msg.latlons()
    
    data_array = xr.DataArray(
        np.array([grib_msg.values]),
        coords={
            "latitude": (["y", "x"], lats),
            "longitude": (["y", "x"], lons),
            "level": [grib_msg.level],
            "x": x,
            "y": y,
        },
        dims=["level", "y", "x"]
    )
    
    return data_array
        

In [13]:
def read_grib(grib_file, variable_names=[], short_names={}):
    variables = {}
    
    for msg in grib_file:
        name = short_names.get(msg.parameterName, "unknown") if msg.shortName == "unknown" else msg.shortName
        
        if len(variable_names) and name not in variable_names:
            continue        
            
        data_array = to_data_array(msg)

        if name not in variables:
            variables[name] = data_array
            continue
            
        variables[name] = xr.concat([variables[name], data_array], dim="level")
        
    return xr.Dataset(variables)

In [14]:
with pygrib.open("data/2132119000017") as grib_file:
    ds = read_grib(grib_file.select(typeOfLevel="hybrid"), ["pres", "gh", "massden", "t"], {"Mass density": "massden"})
    print(ds)

<xarray.Dataset>
Dimensions:    (y: 1059, x: 1799, level: 50)
Coordinates:
    latitude   (y, x) float64 21.14 21.15 21.15 21.16 ... 47.86 47.85 47.84
    longitude  (y, x) float64 -122.7 -122.7 -122.7 ... -60.99 -60.95 -60.92
  * level      (level) int64 1 2 3 4 5 6 7 8 9 10 ... 42 43 44 45 46 47 48 49 50
  * x          (x) float64 -2.698e+06 -2.695e+06 ... 2.693e+06 2.696e+06
  * y          (y) float64 -1.587e+06 -1.584e+06 ... 1.584e+06 1.587e+06
Data variables:
    pres       (level, y, x) float64 1.018e+05 1.018e+05 ... 1.731e+03 1.731e+03
    gh         (level, y, x) float64 10.76 10.76 10.76 ... 2.692e+04 2.692e+04
    t          (level, y, x) float64 294.1 294.1 294.1 ... 210.0 210.0 210.0
    massden    (level, y, x) float64 0.164 0.164 0.162 0.16 ... 0.0 0.0 0.0 0.0


In [15]:
ds.to_netcdf("data/test.nc")

In [16]:
ds.to_zarr("data/test.zarr")

## Benchmarks

In [17]:
from timeit import timeit

In [18]:
def load_test_grib():
    with pygrib.open("data/2132119000017") as grib_file:
        read_grib(grib_file.select(typeOfLevel="hybrid"), ["pres", "gh", "massden", "t"], {"Mass density": "massden"})

In [19]:
timeit(load_test_grib, number=1)

179.29263109100066

In [22]:
def load_test_nc():
    xr.open_dataset("data/test.nc")

In [23]:
timeit(load_test_nc, number=1)

0.4642456149995269

In [24]:
def load_test_zarr():
    xr.open_zarr("data/test.zarr")

In [25]:
timeit(load_test_zarr, number=1)

0.042625548001524294